In [2]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Load the CSV file
# image_data = pd.read_csv("image_data.csv")
image_data = pd.read_csv("image_data.csv")

# Convert integer labels to string
image_data['is_healthy'] = image_data['is_healthy'].astype(str)

# Split the data into training and testing sets
train_data, test_data = train_test_split(image_data, test_size=0.2, random_state=42)

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Create ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create train and test generators
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='image_url',
    y_col='is_healthy',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='image_url',
    y_col='is_healthy',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=test_generator
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Found 16441 validated image filenames belonging to 2 classes.
Found 4111 validated image filenames belonging to 2 classes.
Epoch 1/5
514/514 [==============================] - 3288s 6s/step - loss: 0.2078 - accuracy: 0.9186 - val_loss: 0.1585 - val_accuracy: 0.9579
Epoch 2/5
514/514 [==============================] - 3233s 6s/step - loss: 0.0861 - accuracy: 0.9695 - val_loss: 0.1016 - val_accuracy: 0.9681
Epoch 3/5
514/514 [==============================] - 3435s 7s/step - loss: 0.0698 - accuracy: 0.9759 - val_loss: 0.1779 - val_accuracy: 0.9341
Epoch 4/5
514/514 [==============================] - 817s 2s/step - loss: 0.0412 - accuracy: 0.9864 - val_loss: 0.0338 - val_accuracy: 0.9900
Epoch 5/5
129/129 [==============================] - 43s 329ms/step - loss: 0.0694 - accuracy: 0.9813
Test accuracy: 98.13%


In [3]:
# Save the model
model.save("trained_model.h5")
print("Model saved successfully!")

C:\Users\laksh\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully!
